In [1]:
from vm.mnemonic import WProtectMnemonic
from vm.analysis import PEAnalysis, WProtectEmulator, WProtectTracer
import vm.obfuscation
import semantic.flow
import semantic.translator
from vm.rule_generator import generate_rules

__Load binary file and recover its instructions__  
There are several several samples provided. Those samples test conditional jumps as they do require a context. Uncomment a sample, or provide a binary of your own.

In [2]:
binary = WProtectEmulator("if_all/if_jz.wp.exe")
#binary = WProtectEmulator("if_all/if_jb.wp.exe")
#binary = WProtectEmulator("if_all/if_jo.wp.exe")
#binary = WProtectEmulator("if_all/if_js.wp.exe")
#binary = WProtectEmulator("if_all/if_jbe.wp.exe")
#binary = WProtectEmulator("if_all/if_jl.wp.exe")
#binary = WProtectEmulator("if_all/if_jle.wp.exe")
#binary = WProtectEmulator("if_all/if_jp.wp.exe")
binary.find()
instructions = binary.recover_mnemonics(binary.offset, WProtectMnemonic)[1]
WProtectMnemonic.consolidate_names(instructions)

[INFO]	PE loaded
[INFO]	reachable code analysis done


[INFO]	WProtect vm mnemonics found at offset 0x40a901


[INFO]	Mnemonics loaded


__Reconstruct the control flow and get continuous code blocks.__  
The function _callback_ passes instructions to _WProtectControlFlow_ as they are read by _WProtectTracer_. _WProtectTracer_ always creates new states (one if no branching occured, two otherwise), these states are inserted into a processing queue. In other words, after at most 2 instructions another branch, if available, is processed.  

_WProtectControlFlow_ is then utilised to reconstruct the control flow and output code blocks (a sequence of instructions that is always executed continuously, the only jump is permitted at its end; similarly, the only jump destination is permitted at its beginning).

In [3]:
def callback(offset, instruction, args, off):
    control_flow.add_node(offset, instruction, args, off)

control_flow = semantic.flow.WProtectControlFlow(instructions)
instance = WProtectTracer(binary, instructions)

machine = [instance]
index = 0
while index < len(machine):
    try:
        tmp = machine[index].step(callback)
        machine += tmp
    except semantic.flow.PositionInstructionError as err:
        pass
    index += 1
    
blocks = control_flow.compile_blocks()

__Translate blocks and assemble__  
Use _semantic.translator.translate_blocks_ to translate all the instruction into the assembly language. A label is inserted at the beginning of every block. Translated blocks are concatenated, prepended with a memory reserved for registers, and a jump at the entry point is inserted before the first block.

In [4]:
def get_code(blocks):
    def hexify(string):
        return "".join([hex(ord(char))[2:].zfill(2) for char in string])
    return hexify(semantic.translator.translate_blocks(blocks))

get_code(blocks)

reg_0: .byte 0
reg_1: .byte 0
reg_2: .byte 0
reg_3: .byte 0
reg_4: .byte 0
reg_5: .byte 0
reg_6: .byte 0
reg_7: .byte 0
reg_8: .byte 0
reg_9: .byte 0
reg_10: .byte 0
reg_11: .byte 0
reg_12: .byte 0
reg_13: .byte 0
reg_14: .byte 0
reg_15: .byte 0
reg_16: .byte 0
reg_17: .byte 0
reg_18: .byte 0
reg_19: .byte 0
reg_20: .byte 0
reg_21: .byte 0
reg_22: .byte 0
reg_23: .byte 0
reg_24: .byte 0
reg_25: .byte 0
reg_26: .byte 0
reg_27: .byte 0
reg_28: .byte 0
reg_29: .byte 0
reg_30: .byte 0
reg_31: .byte 0
reg_32: .byte 0
reg_33: .byte 0
reg_34: .byte 0
reg_35: .byte 0
reg_36: .byte 0
reg_37: .byte 0
reg_38: .byte 0
reg_39: .byte 0
reg_40: .byte 0
reg_41: .byte 0
reg_42: .byte 0
reg_43: .byte 0
reg_44: .byte 0
reg_45: .byte 0
reg_46: .byte 0
reg_47: .byte 0
reg_48: .byte 0
reg_49: .byte 0
reg_50: .byte 0
reg_51: .byte 0
reg_52: .byte 0
reg_53: .byte 0
reg_54: .byte 0
reg_55: .byte 0
reg_56: .byte 0
reg_57: .byte 0
reg_58: .byte 0
reg_59: .byte 0
reg_60: .byte 0
reg_61: .byte 0
reg_62: .byte 0
re

'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000e9dc01000058a35800000058a32800000058a31800000058a33400000058a33c00000058a36000000058a33800000058a35c00000058a300000000a12800000050b80100000050595801c8509cb8fefbffff5058a34800000054588b005058f7d05bf7d39c58a354000000a1480000005054588b005058f7d05bf7d39c58a34c00000058f7d05bf7d39c58a330000000a15800000050b8010400005058a34800000054588b005058f7d05bf7d39c58a314000000a1480000005054588b005058f7d05bf7d39c58a31c00000058f7d05bf7d39c58a35000000058a34800000058a314000000a14800000050a1140000005058f7d05bf7d39c58a330000000a14800000050a1140000005058f7d05bf7d39c58a34000000058f7d05bf7d39c58a30c00000058a35800000058a